#### Installation & Imports

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Database tools
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# AI Models
from langchain_google_genai import ChatGoogleGenerativeAI

# Learning tools
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate


#### Configuration & Credentials

In [2]:
# Look for the .env file one folder up (in 'gen_ai')
env_path = Path('..') / '.env'

# Force reload to ensure we have the latest keys
load_dotenv(dotenv_path=env_path, override=True)

# Grab the specific keys
api_key = os.getenv("GOOGLE_API_KEY")
db_user = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")
db_port = os.getenv("DB_PORT")

if not api_key:
    print("❌ Error: API Key not found. Check your .env path!")
else:
    print(f"✅ Keys loaded! Ready to connect to: {db_host}")

✅ Keys loaded! Ready to connect to: gateway01.eu-central-1.prod.aws.tidbcloud.com


#### Connect to Database

In [3]:
# We use mysql-connector for secure TiDB Cloud connections
db_uri = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

print(f"🔌 Connecting to database...")

try:
    # 'pool_pre_ping' keeps the connection from dying if you are idle
    engine = create_engine(db_uri, pool_pre_ping=True, pool_recycle=300)
    
    # Connect LangChain to the SQL engine
    db = SQLDatabase(engine, sample_rows_in_table_info=3)
    
    print("✅ Connected successfully! Here is your database structure:")
    print(db.table_info)

except Exception as e:
    print("❌ Connection Failed. Check your internet or password.")
    print(e)

🔌 Connecting to database...
✅ Connected successfully! Here is your database structure:

CREATE TABLE atliq_tshirts.t_shirts (
	t_shirt_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER(11), 
	stock_quantity INTEGER(11) NOT NULL, 
	PRIMARY KEY (t_shirt_id)
)COLLATE utf8mb4_bin DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	M	35	80
2	Nike	White	S	40	50
3	Adidas	Black	L	45	30
*/


CREATE TABLE discounts (
	discount_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER(11) NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT fk_1 FOREIGN KEY(t_shirt_id) REFERENCES atliq_tshirts.t_shirts (t_shirt_id)
)COLLATE utf8mb4_bin DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_i

#### Define "Few-Shot" Examples

In [4]:
few_shots = [
    {'Question': "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer': "91"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "22292"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
     'SQLQuery': "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id",
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"},
    {'Question': "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
     'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
     'SQLResult': "Result of the SQL query",
     'Answer': "17462"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer': "50"}
]

#### Setup Embeddings

In [5]:
print("🧠 Building the Knowledge Base...")

# Use CPU to prevent crashes
model_kwargs = {'device': 'cpu'} 
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Store examples in ChromaDB (The "Smart Filing Cabinet")
to_vectorize = [" ".join(example.values()) for example in few_shots]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

# Create the selector that picks the top 2 most relevant examples
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)
print("✅ Knowledge Base Ready!")

🧠 Building the Knowledge Base...
✅ Knowledge Base Ready!


#### Define Prompts

In [10]:
# Main Instruction Prompt
# We added a line at the bottom: "Important: ..." to stop the repetition.
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

Important: In the final 'Answer' section, do not repeat the SQL query. Just explain the result to the user in a natural sentence.
"""

# Suffix
PROMPT_SUFFIX = """Only use the following tables:
{table_info}

Question: {input}"""

# Assemble Prompt Template
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

print("✅ Updated Instructions: The AI will now be less robotic.")

✅ Updated Instructions: The AI will now be less robotic.


#### Run the Chain

In [11]:
print("🚀 Initializing Gemini Flash (Latest)...")

# We use the alias that explicitly appeared in your "Available Models" list.
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest", 
    google_api_key=api_key, 
    temperature=0.1
)

# Connect the Brain (LLM) to the Database
chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

# Run the Test
print("\n--- TEST QUERY ---")
try:
    # Use .invoke() to avoid warnings
    response = chain.invoke("How many white Nike t-shirts do we have?")
    print(f"\n✅ Final Answer: {response['result']}")
    
except Exception as e:
    print(f"❌ Error: {e}")

🚀 Initializing Gemini Flash (Latest)...

--- TEST QUERY ---


> Entering new SQLDatabaseChain chain...
How many white Nike t-shirts do we have?
SQLQuery:Question: How many white Nike t-shirts do we have?
SQLQuery: SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(Decimal('50'),)]
Answer:Question: How many white Nike t-shirts do we have?
SQLQuery: SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
> Finished chain.

✅ Final Answer: Question: How many white Nike t-shirts do we have?
SQLQuery: SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White'


In [12]:
question = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"

response = chain.invoke(question)

print(f"\nAnswer: {response['result']}")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('4320.000000'),)]
Answer:Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = '